<a href="https://colab.research.google.com/github/lennart194/thesis-code/blob/main/pooling_rf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ddop

In [2]:
import numpy as np
import pandas as pd

from ddop.newsvendor import RandomForestWeightedNewsvendor
from ddop.newsvendor import SampleAverageApproximationNewsvendor

from sklearn.model_selection import RandomizedSearchCV

from ddop.metrics import average_costs
from ddop.metrics import prescriptiveness_score
from ddop.metrics import make_scorer

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## loading data-set


---



*   load the whole ultimative set
*   sort the rows first by item, then by date and set a multi index
**now every items time series can be separated by the index**

In [4]:
data = pd.read_csv('/content/drive/MyDrive/M5/ultimative_set.csv')
data = data.drop(columns=['Unnamed: 0'])
data = data.sort_values(by=['item_id', 'date'])
data = data.set_index(['item_id', 'date'])

## split in feature and target matrix


---

In [5]:
X = data.drop(columns=['demand'])
Y = data['demand']

## train_test_split


---

In [6]:
X_train = X.loc[(slice(None), slice(None, "2015")), :]
X_test = X.loc[(slice(None), slice("2015", '2017')), :]

Y_train = Y.loc[(slice(None), slice(None, "2015"))]
Y_test = Y.loc[(slice(None), slice("2015", "2017"))]

## defining the different cost paramerters for special service level and product range



---



*   cu is the mean of the cumulated 'sell_price' column of the training data
*   appending to the corresponding service level co is calculated with (cu - α * cu) / α
*   all the costs are rounded to 2 digits

In [7]:
prods = list(range(0,25))

cu = round(X_train['sell_price'].mean(),2)

co_90 = round(((cu-(0.90*cu))/0.90),2)

co_75 = round(((cu-(0.75*cu))/0.75),2)

co_50 = round(((cu-(0.50*cu))/0.50),2)

## determining some parameters


---


*   the potential range of the params for randomized search are defined
*   the estimator, the scorer and finally the whole param_random is defined

In [ ]:
max_depth = np.arange(7, 24)
n_estimators = np.arange(65, 101)
min_samples_leaf = np.arange(2, 6)
param_random = dict(max_depth = max_depth, n_estimators = n_estimators, min_samples_leaf = min_samples_leaf)

scorer_avc = make_scorer(average_costs, greater_is_better=False)

# rf for 90% service level


---






## randomized search




*   randomized serach is done once for the entire data-set with the corresponding service level costs
*   this step was done with the server of the University of Würzburg because of the high resource consumption 
*   the results for the best params are: n_estimators = 79 , max_depth = 21 , min_samples_leaf = 2

In [ ]:
rf = RandomForestWeightedNewsvendor(cu = cu, co = co_90, random_state = 42, criterion = 'mse')
random = RandomizedSearchCV(rf, param_random, cv=5, scoring = scorer_avc, n_jobs=8)
random.fit(X_train, Y_train)
print(random.best_params_)

## newsvendor-model 90%



---



*   the model is trained and tested once for entire data-set with the best hyperparameters
*   the prediction is the output

In [8]:
rf = RandomForestWeightedNewsvendor(cu = cu, co = co_90, random_state = 42, criterion = 'mse',
                                    max_depth = 21, n_estimators = 79, min_samples_leaf = 2)

rf.fit(X_train, Y_train)
preds_90 = rf.predict(X_test)

## reorder the data




---



*   create a pandas frame 'comp_90' where the real values, the predictions and the predictions from the saa model are in columns next to each other

In [9]:
pred_90 = pd.DataFrame(preds_90, columns=['pred_rf_90'])
comp_90 = Y.loc[(slice(None), slice("2015", "2017"))].reset_index().sort_values(by=['date', 'item_id']).set_index('date')
comp_90 = comp_90.reset_index()
comp_90['pred_rf_90'] = pred_90[['pred_rf_90']]
comp_90 = comp_90.sort_values(by=['item_id', 'date']).set_index(['item_id', 'date'])

saa_90 = pd.read_csv('/content/drive/MyDrive/M5/SAA_pred_90.csv').set_index(['item_id', 'date'])

comp_90 = pd.concat([comp_90, saa_90], axis = 1)

## average costs and prescriptivess score


---



*   create a loop where the average_costs (the prescriptivess score) of each item are calculated

*   save the results into a pandas frame


In [11]:
result_rf_90_avc = []

for prod in prods:
  avc = round(average_costs(comp_90['demand'].loc[prod], comp_90['pred_rf_90'].loc[prod], cu = cu, co = co_90),2)
  result_rf_90_avc.append(avc)

In [12]:
RF_pooled_avc_90 = pd.DataFrame(result_rf_90_avc, columns=['RF_avc_90'])
RF_pooled_avc_90.index.name = 'item_id'

In [14]:
result_rf_90_pscr = []

for prod in prods:
  pscr = round(prescriptiveness_score(comp_90['demand'].loc[prod], comp_90['pred_rf_90'].loc[prod],
                                      comp_90['SAA_pred_90'].loc[prod], cu = cu, co = co_90),2)
  result_rf_90_pscr.append(pscr)

In [15]:
RF_pooled_pscr_90 = pd.DataFrame(result_rf_90_pscr, columns=['RF_pscr_90'])
RF_pooled_pscr_90.index.name = 'item_id'

# rf for 75% service level


---






## randomized search




*   randomized serach is done once for the entire data-set with the corresponding service level costs
*   this step was done with the server of the University of Würzburg because of the high resource consumption 
*   the results for the best params are: n_estimators = 87 , max_depth = 18 , min_samples_leaf = 2

In [ ]:
rf = RandomForestWeightedNewsvendor(cu = cu, co = co_75, random_state = 42, criterion = 'mse')
random = RandomizedSearchCV(rf, param_random, cv=5, scoring = scorer_avc, n_jobs=8)
random.fit(X_train, Y_train)
print(random.best_params_)

## newsvendor-model 75%



---



*   the model is trained and tested once for entire data-set with the best hyperparameters
*   the prediction is the output

In [20]:
rf = RandomForestWeightedNewsvendor(cu = cu, co = co_75, random_state = 42, criterion = 'mse',
                                    max_depth = 18, n_estimators = 87, min_samples_leaf = 2)

rf.fit(X_train, Y_train)
preds_75 = rf.predict(X_test)

## reorder the data




---



*   create a pandas frame 'comp_75' where the real values, the predictions and the predictions from the saa model are in columns next to each other

In [21]:
pred_75 = pd.DataFrame(preds_75, columns=['pred_rf_75'])
comp_75 = Y.loc[(slice(None), slice("2015", "2017"))].reset_index().sort_values(by=['date', 'item_id']).set_index('date')
comp_75 = comp_75.reset_index()
comp_75['pred_rf_75'] = pred_75[['pred_rf_75']]
comp_75 = comp_75.set_index(['item_id', 'date'])

saa_75 = pd.read_csv('/content/drive/MyDrive/M5/SAA_pred_75.csv').set_index(['item_id', 'date'])

comp_75 = pd.concat([comp_75, saa_75], axis = 1)

## average costs and prescriptivess score


---



*   create a loop where the average_costs (the prescriptivess score) of each item are calculated

*   save the results into a pandas frame

In [24]:
result_rf_75_avc = []

for prod in prods:
  avc = round(average_costs(comp_75['demand'].loc[prod], comp_75['pred_rf_75'].loc[prod], cu = cu, co = co_75),2)
  result_rf_75_avc.append(avc)

In [25]:
RF_pooled_avc_75 = pd.DataFrame(result_rf_75_avc, columns=['RF_avc_75'])
RF_pooled_avc_75.index.name = 'item_id'

In [26]:
result_rf_75_pscr = []

for prod in prods:
  pscr = round(prescriptiveness_score(comp_75['demand'].loc[prod], comp_75['pred_rf_75'].loc[prod],
                                      comp_75['SAA_pred_75'].loc[prod], cu = cu, co = co_75),2)
  result_rf_75_pscr.append(pscr)

In [27]:
RF_pooled_pscr_75 = pd.DataFrame(result_rf_75_pscr, columns=['RF_pscr_75'])
RF_pooled_pscr_75.index.name = 'item_id'

# rf for 50% service level


---






## randomized search




*   randomized serach is done once for the entire data-set with the corresponding service level costs
*   this step was done with the server of the University of Würzburg because of the high resource consumption 
*   the results for the best params are: n_estimators = 93 , max_depth = 18 , min_samples_leaf = 5

In [ ]:
rf = RandomForestWeightedNewsvendor(cu = cu, co = co_50, random_state = 42, criterion = 'mse')
random = RandomizedSearchCV(rf, param_random, cv=5, scoring = scorer_avc, n_jobs=8)
random.fit(X_train, Y_train)
print(random.best_params_)

## newsvendor-model 50%



---



*   the model is trained and tested once for entire data-set with the best hyperparameters
*   the prediction is the output

In [30]:
rf = RandomForestWeightedNewsvendor(cu = cu, co = co_50, random_state = 42, criterion = 'mse',
                                    max_depth = 18, n_estimators = 93, min_samples_leaf = 5)

rf.fit(X_train, Y_train)
preds_50 = rf.predict(X_test)

## reorder the data




---



*   create a pandas frame 'comp_50' where the real values, the predictions and the predictions from the saa model are in columns next to each other

In [31]:
pred_50 = pd.DataFrame(preds_50, columns=['pred_rf_50'])
comp_50 = Y.loc[(slice(None), slice("2015", "2017"))].reset_index().sort_values(by=['date', 'item_id']).set_index('date')
comp_50 = comp_50.reset_index()
comp_50['pred_rf_50'] = pred_50[['pred_rf_50']]
comp_50 = comp_50.set_index(['item_id', 'date'])

saa_50 = pd.read_csv('/content/drive/MyDrive/M5/SAA_pred_50.csv').set_index(['item_id', 'date'])

comp_50 = pd.concat([comp_50, saa_50], axis = 1)

## average costs and prescriptivess score


---



*   create a loop where the average_costs (the prescriptivess score) of each item are calculated

*   save the results into a pandas frame

In [33]:
result_rf_50_avc = []

for prod in prods:
  avc = round(average_costs(comp_50['demand'].loc[prod], comp_50['pred_rf_50'].loc[prod], cu = cu, co = co_50),2)
  result_rf_50_avc.append(avc)

In [34]:
RF_pooled_avc_50 = pd.DataFrame(result_rf_50_avc, columns=['RF_avc_50'])
RF_pooled_avc_50.index.name = 'item_id'

In [35]:
result_rf_50_pscr = []

for prod in prods:
  pscr = round(prescriptiveness_score(comp_50['demand'].loc[prod], comp_50['pred_rf_50'].loc[prod],
                                      comp_50['SAA_pred_50'].loc[prod], cu = cu, co = co_50),2)
  result_rf_50_pscr.append(pscr)

In [36]:
RF_pooled_pscr_50 = pd.DataFrame(result_rf_50_pscr, columns=['RF_pscr_50'])
RF_pooled_pscr_50.index.name = 'item_id'

# merge the pandas frames


---

In [37]:
RF_pooled_avc = pd.concat([RF_pooled_avc_90, RF_pooled_avc_75, RF_pooled_avc_50], axis=1)
RF_pooled_avc

,RF_avc_90,RF_avc_75,RF_avc_50
item_id,,,
0,5.81,11.06,21.33
1,4.48,10.93,25.61
2,5.99,11.04,21.28
3,6.07,11.22,21.65
4,9.57,14.71,24.12
5,4.35,9.96,22.36
6,6.97,11.96,21.75
7,6.18,11.11,21.10
8,4.26,9.70,21.21


In [38]:
RF_pooled_pscr = pd.concat([RF_pooled_pscr_90, RF_pooled_pscr_75, RF_pooled_pscr_50], axis=1)
RF_pooled_pscr

,RF_pscr_90,RF_pscr_75,RF_pscr_50
item_id,,,
0,-0.56,-0.48,-0.69
1,-1.50,-1.77,-3.06
2,-1.05,-0.97,-1.30
3,-0.15,-0.10,-0.41
4,-1.96,-1.38,-1.17
5,-1.09,-1.36,-2.49
6,-1.09,-0.69,-0.83
7,-1.26,-1.31,-1.63
8,-1.20,-1.62,-2.31


# save the results


---

In [39]:
RF_pooled_avc.to_csv('/content/drive/MyDrive/M5/RF_pooled_avc.csv')

RF_pooled_pscr.to_csv('/content/drive/MyDrive/M5/RF_pooled_pscr.csv')